In [1]:
import warnings
import numpy as np

from glob import glob
from tqdm import tqdm

from skimage.io import imsave, imread
from skimage import img_as_ubyte, exposure, filters
from skimage.color import rgb2gray

from commons import (load_images_from_paths, 
                     load_paths_images_sorted,
                     binarize_image,
                     auto_invert_image_mask,
                     find_bighest_cluster,
                     check_colision_border,
                     crop_image_box,
                     smoothing_mask_edges,
                     find_center_mask,
                     find_roi,
                     plot)

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def segmenter_v1(frame,_):
    
    """
        Menos performático
    """
    mask = find_roi(frame)
    mask = smoothing_mask_edges(mask)
    mask = auto_invert_image_mask(mask)

    return mask

def segmenter_v2(frame, background):
    
    """
        Mais performático
    """
    mask = binarize_image(np.subtract(exposure.equalize_hist(frame), 
                                      exposure.equalize_hist(background)))
    mask = find_bighest_cluster(mask)
    mask = smoothing_mask_edges(mask)
    
    mask = auto_invert_image_mask(mask)
    
    return mask

def segmenter_v3(frame, background):
    """
        Mediano
    """
    
    mask = binarize_image(np.subtract(frame, background))
    mask = auto_invert_image_mask(mask)
    
    mask = find_bighest_cluster(mask)
    mask = smoothing_mask_edges(mask)
    
    mask = auto_invert_image_mask(mask)
    
    return mask

# Rodando modelos de CROP

In [4]:
def run_segmenter(path_in,
                  path_out, 
                  margin_pixel=100,
                  func_segment=None):
    
    try:
        import os
        
        image_name = path_in.split('/')[-1]
        
        path_image_out = f'{path_out}/images/{image_name}'
        path_masks_out = f'{path_out}/masks/{image_name}'
        
        os.makedirs(path_masks_out)
        os.makedirs(path_image_out)
    except Exception as e:
        print(e)
        pass

    frames_sorted = load_paths_images_sorted(path_in)
    background, * frames = load_images_from_paths(frames_sorted, is_gray=True)
    _, * frames_paths = frames_sorted

    for path, frame in  zip(frames_paths, tqdm(frames)):

        try:
            img_name = path.split('/')[-1]

            mask = func_segment(frame, background)

            if check_colision_border(mask):
                continue
                
            center_coords = find_center_mask(mask)
            mask_croped = crop_image_box(mask, 
                                         center_coords,
                                         margin_pixel)
                
            frame_croped = crop_image_box(frame, 
                                          center_coords, 
                                          margin_pixel)
                
            imsave(f'{path_image_out}/{img_name}', img_as_ubyte(frame_croped))
            imsave(f'{path_masks_out}/{img_name}', img_as_ubyte(mask_croped.astype(dtype=bool)))
        except Exception as e: 
            continue

In [5]:
for path in sorted(glob("dataset/*")):
    run_segmenter(path_in=path,
                  path_out="segmenter_v2",
                  margin_pixel=110,
                  func_segment=segmenter_v2)   

100%|███████████████████████████████████████████████████████████████| 429/429 [00:18<00:00, 23.13it/s]
